In [7]:
from neuralNetwork import *
import scipy.io as scio
import matplotlib.pyplot as plt
import time
import csv
from numba import jit

In [8]:
newData = scio.loadmat('originalData/letters_data.mat')
trainYencoded = scio.loadmat('trainYencoded.mat')
# I converted the labels in MATLAB using the included encodeLabels.m
# since it was much faster than Python.
# I simply called encodeLabels on the original train_y set.

trainX = newData['train_x']
trainY = trainYencoded['trainY']
testX = newData['test_x']

NN = NeuralNetwork(784,200,26,0.01)

In [9]:
NN.V = (NN.V + abs(NN.V.min()))
NN.W = (NN.W + abs(NN.W.min()))

In [10]:
""" Adjust learning rate """

NN.learningRate = 0.01

In [11]:
""" ----- General Training Function ----- """

def runTraining(numEpochs=300):
    currEpoch = 0
    while currEpoch < numEpochs:
        print("\nCurrent Epoch: ",currEpoch)
        indices = range(0, 124800)
        indices = np.random.permutation(indices)

        j = 0
        cost = 0

        while j < 124800:
            i = indices[j]
            X = np.append(trainX[i,:], 1)
            X = (X - np.mean(X))/np.std(X)

            NN.trainNetwork(X, trainY[i])
            cost += NN.crossEntropyError(X, trainY[i])
            if j % 5000 == 0 and j != 0:
                print(cost / j)

            j += 1

        currEpoch += 1
        NN.learningRate = NN.learningRate*0.99

In [ ]:
runTraining()


Current Epoch:  0
1.33368309048
0.991636540538
0.839859823336
0.748062572939
0.687485861464
0.643117979746
0.609624266167
0.583642442185
0.56441376871
0.548678816106
0.534118312895
0.521896712822
0.510751769071
0.500394192771
0.492152167864
0.484979028303
0.478147027778
0.471718887777
0.466645728363
0.46309303885
0.45808982042
0.454502593523
0.451709650808
0.448360577192

Current Epoch:  1
0.361197930496
0.353096529937
0.35707241407
0.355077423088
0.35403990583
0.355362509297
0.354311120292
0.353521949549
0.35445279709
0.355971987066
0.356089113811
0.355598184803
0.356232736414
0.357529302334
0.357972062035
0.357039294995
0.357773854963
0.357661241651
0.357334524634
0.35691991276
0.35646168766
0.356071133447
0.355864906691
0.355911226135

Current Epoch:  2
0.342438611788
0.341726140414
0.335665877828
0.33830027386
0.340429506892
0.338608843189
0.340413202785
0.341930060538
0.342118792294
0.34037861641
0.339586380298
0.33841062354


In [ ]:
""" Cross Validation Training """
startTime = time.time()
currEpoch = 0
recordCost = []
iterations = []
while currEpoch < 1:
    print("\nCurrent Epoch: ",currEpoch)
    indices = range(0, 124800)
    indices = np.random.permutation(indices)

    j = 0
    cost = 0

    while j < 99840:
        i = indices[j]
        X = np.append(trainX[i,:], 1)
        X = (X - np.mean(X))/np.std(X)
        
        NN.trainNetwork(X, trainY[i])
        cost += NN.crossEntropyError(X, trainY[i])
        if j % 2000 == 0 and j != 0:
            iterations.append(j)
            recordCost.append(cost / j)
            print(cost / j)

        j += 1

    currEpoch += 1
    NN.learningRate = NN.learningRate*0.99

In [ ]:
plt.plot(iterations,recordCost)
plt.show()

In [ ]:
""" ----- Cross Validation Results (20% of Data) ----- """

correct = 0
validationSize = 124800 - j
print(j,validationSize)
fiveCorrect = []
fiveWrong = []
while j < 124800:
    i = indices[j]
    X = np.append(trainX[i,:], 1) # (785,1) np array
    X = (X - np.mean(X))/np.std(X)
    label = np.argmax(trainY[i])
    output = np.argmax(NN.predict(X))
    #print(label,output,label-output)
    
    if label == output:
        correct += 1
        
        if len(fiveCorrect) < 5:
            fiveCorrect.append(trainX[i,:].reshape(28,28))
    else:
        if len(fiveWrong) < 5:
            fiveWrong.append(trainX[i,:].reshape(28,28))
        
    j += 1

endTime = time.time()
total_time = endTime-startTime

print("Validation Accuracy: " + str(correct/validationSize) + " | Training time (s): " + str(total_time))


In [ ]:
""" ----- Final Test Results ----- """
testX = newData['test_x']
k = 0
predLabels = []
while k < 20800:
    X = np.append(testX[k,:], 1) # (785,1) np array
    X = (X - np.mean(X))/np.std(X)
    output = np.argmax(NN.predict(X))
    predLabels.append(output)
    
    k += 1

In [ ]:
""" ----- Save Predicted Labels (Kaggle) ----- """
i = 0
j = 1
with open('results.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=' ', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['Id,Category'])
    for label in predLabels:
        filewriter.writerow([str(j) + ',' + str(label+1)])
        i += 1
        j += 1

In [ ]:
""" ----- Visualization (Problem 3) ----- """
print("5 Correct Results:")
for correctPred in fiveCorrect:
    plt.imshow(correctPred)
    plt.show()

In [ ]:
print("5 Wrong Results:")
for wrongPred in fiveWrong:
    plt.imshow(wrongPred)
    plt.show()

In [ ]:
""" Test speed of training """

%prun runTraining()